In [51]:
# Install the libraries in your kernel
%pip install pandas
%pip install scikit-learn
%pip install xgboost
%pip install plotly


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [52]:
# Make the necessary imports

import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
import xgboost as xgb
from sklearn.preprocessing import LabelEncoder
import plotly.express as px
#import chart_studio.plotly as py

In [8]:
# Read the (training data)

genre_df = pd.read_csv('subgenre.csv')

In [9]:
# A helper dataframe used to make a visualization. It counts the number of albums in a given genre

count_df = genre_df.groupby('genre').count().sort_values(by='url',ascending=False).head(25).reset_index()[['genre','url']]
count_df['count'] = count_df['url']

In [46]:
c = dict(zip(count_df['genre'],px.colors.qualitative.Plotly))

In [53]:
# Creates a pie chart in Plotly

fig = px.pie(count_df.head(15),values='count',names='genre',hole=.3, labels=count_df['genre'],height=600,color_discrete_map=c,color='genre')
fig.update_layout(title_text = 'Breakdown of Subgenres of Jazz Present in Web Crawl')
fig.update_traces(textinfo='label+percent')
fig.show()
#py.plot(fig,filename=f'jazz-subgrenres',auto_open=True)


'https://plotly.com/~jv24601/44/'

[Parent 348738, IPC I/O Parent] WARNING: process 349191 is a zombie: file /builds/worker/checkouts/gecko/ipc/chromium/src/base/process_util_posix.cc:245
[Parent 348738, IPC I/O Parent] WARNING: process 349103 is a zombie: file /builds/worker/checkouts/gecko/ipc/chromium/src/base/process_util_posix.cc:245
[Parent 348738, IPC I/O Parent] WARNING: process 349262 is a zombie: file /builds/worker/checkouts/gecko/ipc/chromium/src/base/process_util_posix.cc:245
[Parent 348738, IPC I/O Parent] WARNING: process 348832 is a zombie: file /builds/worker/checkouts/gecko/ipc/chromium/src/base/process_util_posix.cc:245
[Parent 348738, IPC I/O Parent] WARNING: process 349084 is a zombie: file /builds/worker/checkouts/gecko/ipc/chromium/src/base/process_util_posix.cc:245
[Parent 348738, IPC I/O Parent] WARNING: process 348949 is a zombie: file /builds/worker/checkouts/gecko/ipc/chromium/src/base/process_util_posix.cc:245
[Parent 348738, IPC I/O Parent] WARNING: process 349090 is a zombie: file /builds/

In [11]:
# Translates the text of the wikipedia articles for our training data into np vectors using tfidf 

vectorizer = TfidfVectorizer(max_features=1500,stop_words='english',min_df =.05,max_df=.95)
vectors = vectorizer.fit_transform(genre_df['text']) 

In [12]:
# Translates the genres into numerical categories

le = LabelEncoder()
y_encoded = le.fit_transform(genre_df['genre'])

In [13]:
# Specify our training data and training labels for xgboost

data_dmatrix = xgb.DMatrix(data=vectors,label=y_encoded)
params = {'random_state':'42','num_class':len(count_df)}

In [14]:
# Running cross-validation for our XGB Classifier model gives a good idea of our model's accuracy

xgb_cv = xgb.cv(dtrain=data_dmatrix,params=params, nfold=5,metrics='merror')
xgb_cv

,train-merror-mean,train-merror-std,test-merror-mean,test-merror-std
0,0.138323,0.015297,0.303777,0.028985
1,0.070627,0.005687,0.281374,0.029540
2,0.036780,0.002985,0.239783,0.039394
3,0.021320,0.003563,0.220600,0.034815
4,0.012259,0.003986,0.213147,0.030616
5,0.005863,0.002610,0.212061,0.037875
6,0.003732,0.002445,0.209910,0.043018
7,0.001866,0.001360,0.201399,0.037283
8,0.001066,0.000998,0.197127,0.039852
9,0.000533,0.000653,0.199260,0.039221


In [15]:
model = xgb.XGBClassifier(random_state=42)
model.fit(vectors,y_encoded)


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, objective='multi:softprob', ...)

In [16]:
jazz_df = pd.read_csv('jazz.csv')

In [17]:
jazz_df.head()

,Unnamed: 0,url,title,album_artist,text
0,3,https://en.wikipedia.org/wiki/Miles_Davis_and_...,Miles Davis and the Modern Jazz Giants,Miles Davis,Miles Davis and the Modern Jazz Giants - W...
1,4,https://en.wikipedia.org/wiki/Someday_My_Princ...,Someday My Prince Will Come (Miles Davis album),Miles Davis,Someday My Prince Will Come (Miles Davis a...
2,5,https://en.wikipedia.org/wiki/Miles_%26_Monk_a...,Miles & Monk at Newport,NaN,Miles & Monk at Newport - Wikipedia ...
3,8,https://en.wikipedia.org/wiki/Steamin%27_with_...,Steamin' with the Miles Davis Quintet,Miles Davis,Steamin' with the Miles Davis Quintet - Wi...
4,9,https://en.wikipedia.org/wiki/%27Round_About_M...,'Round About Midnight,Miles Davis,'Round About Midnight - Wikipedia ...


In [18]:
jazz_vectors = vectorizer.transform(jazz_df['text'])

In [19]:
y_pred = model.predict(jazz_vectors)

In [20]:
jazz_df['predicted_genre'] = le.inverse_transform(y_pred)

In [21]:
jazz_df.head()

,Unnamed: 0,url,title,album_artist,text,predicted_genre
0,3,https://en.wikipedia.org/wiki/Miles_Davis_and_...,Miles Davis and the Modern Jazz Giants,Miles Davis,Miles Davis and the Modern Jazz Giants - W...,jazz fusion
1,4,https://en.wikipedia.org/wiki/Someday_My_Princ...,Someday My Prince Will Come (Miles Davis album),Miles Davis,Someday My Prince Will Come (Miles Davis a...,hard bop
2,5,https://en.wikipedia.org/wiki/Miles_%26_Monk_a...,Miles & Monk at Newport,NaN,Miles & Monk at Newport - Wikipedia ...,jazz fusion
3,8,https://en.wikipedia.org/wiki/Steamin%27_with_...,Steamin' with the Miles Davis Quintet,Miles Davis,Steamin' with the Miles Davis Quintet - Wi...,modal jazz
4,9,https://en.wikipedia.org/wiki/%27Round_About_M...,'Round About Midnight,Miles Davis,'Round About Midnight - Wikipedia ...,hard bop


In [22]:
jazz_count_df  = jazz_df.groupby('predicted_genre').count().sort_values(by='text',ascending=False).head(25).reset_index()[['predicted_genre','text']]
jazz_count_df['count'] = jazz_count_df['text']

In [54]:
# Creates a pie chart in Plotly

fig2 = px.pie(jazz_count_df.head(15),values='count',names='predicted_genre',hole=.3, labels=jazz_count_df['predicted_genre'],height=600,color_discrete_map=c,color='predicted_genre')
fig2.update_layout(title_text = 'Breakdown of Predicted Subgenres of Jazz Present in Web Crawl')
fig2.update_traces(textinfo='label+percent')
fig2.show()
#py.plot(fig2,filename=f'jazz-predicted-subgrenres',auto_open=True)


'https://plotly.com/~jv24601/46/'

[Parent 349430, IPC I/O Parent] WARNING: process 349904 is a zombie: file /builds/worker/checkouts/gecko/ipc/chromium/src/base/process_util_posix.cc:245
[Parent 349430, IPC I/O Parent] WARNING: process 349530 is a zombie: file /builds/worker/checkouts/gecko/ipc/chromium/src/base/process_util_posix.cc:245
[Parent 349430, IPC I/O Parent] WARNING: process 349705 is a zombie: file /builds/worker/checkouts/gecko/ipc/chromium/src/base/process_util_posix.cc:245
[Parent 349430, IPC I/O Parent] WARNING: process 349603 is a zombie: file /builds/worker/checkouts/gecko/ipc/chromium/src/base/process_util_posix.cc:245
[Parent 349430, IPC I/O Parent] WARNING: process 349711 is a zombie: file /builds/worker/checkouts/gecko/ipc/chromium/src/base/process_util_posix.cc:245


In [49]:
jazz_df[jazz_df['predicted_genre']=='vocal jazz'].head(20)

,Unnamed: 0,url,title,album_artist,text,predicted_genre
36,121,https://en.wikipedia.org/wiki/Tenor_Conclave,Tenor Conclave,John Coltrane,Tenor Conclave - Wikipedia ...,vocal jazz
80,211,https://en.wikipedia.org/wiki/Lift_Every_Voice...,Lift Every Voice (Andrew Hill album),Andrew Hill,Lift Every Voice (Andrew Hill album) - Wik...,vocal jazz
98,237,https://en.wikipedia.org/wiki/Dizzy_Gillespie_...,Dizzy Gillespie at Newport,Dizzy Gillespie,Dizzy Gillespie at Newport - Wikipedia ...,vocal jazz
138,304,https://en.wikipedia.org/wiki/The_Quintet_(album),The Quintet (album),V.S.O.P.,The Quintet (album) - Wikipedia ...,vocal jazz
152,328,https://en.wikipedia.org/wiki/Ella_Fitzgerald_...,Ella Fitzgerald and Billie Holiday at Newport,NaN,Ella Fitzgerald and Billie Holiday at Newp...,vocal jazz
155,333,https://en.wikipedia.org/wiki/Sunshine_of_Your...,Sunshine of Your Love (album),Ella Fitzgerald,Sunshine of Your Love (album) - Wikipedia ...,vocal jazz
161,343,https://en.wikipedia.org/wiki/Newport_Jazz_Fes...,Newport Jazz Festival: Live at Carnegie Hall,Ella Fitzgerald,Newport Jazz Festival: Live at Carnegie Ha...,vocal jazz
162,344,https://en.wikipedia.org/wiki/Echoes_of_an_Era,Echoes of an Era,Chaka Khan,Echoes of an Era - Wikipedia ...,vocal jazz
163,347,https://en.wikipedia.org/wiki/At_the_Opera_House,At the Opera House,NaN,At the Opera House - Wikipedia ...,vocal jazz
168,353,https://en.wikipedia.org/wiki/Montreux_%2775,Montreux '75,Ella Fitzgerald,Montreux '75 - Wikipedia ...,vocal jazz
